In [1]:
!pip install crewai==0.32.2 crewai_tools==0.3.0 langchain langchain-huggingface


In [8]:
import os
openai_api_key = ""
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [3]:
from crewai import Agent, Task, Crew


In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	# verbose=True
)

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    # verbose=True
)

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    # verbose=True
)

In [11]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [10]:

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [9]:


edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

from IPython.display import Markdown
Markdown(result)

2024-06-26 17:43:00,674 - 139850343515968 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-06-26 17:43:00][DEBUG]: == Working Agent: Content Planner
 [2024-06-26 17:43:00][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


In [4]:
from langchain_huggingface import HuggingFacePipeline


llm = HuggingFacePipeline.from_model_id(
    model_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
     pipeline_kwargs={
        "max_new_tokens": 500,
        "top_k": 50,
        "temperature": 0.1,
    },
)

planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    llm=llm
)

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provided by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provided by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm=llm
)

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm=llm
)

plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
    llm=llm
)

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
    llm=llm
)

edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})
from IPython.display import Markdown
Markdown(result)


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

 [2024-06-26 17:24:24][DEBUG]: == Working Agent: Content Planner
 [2024-06-26 17:24:24][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 [2024-06-26 17:30:08][DEBUG]: == [Content Planner] Task output: Title: The Future of Artificial Intelligence: Trends, Key Players, and Noteworthy News

Introduction:
Artificial Intelligence (AI) is no longer a futuristic concept but a reality that is transforming various industries. From healthcare to finance, AI is making a significant impact on the way we live and work. In this article, we will explore the latest trends, key players, and noteworthy news in the world of AI.

Key Points:
1. Latest Trends:
a. Natural Language Processing (NLP): NLP is a subfield of AI that enables computers to understand and interpret human language. NLP is being used in various applications such as chatbots, virtual assistants, and customer service.
b. Reinforcement Learning: Reinforcement learning is a type of machine learning that allows computers to learn by interacting with their environment. Reinforcement learning is being used in various applications such as gaming, robotics, and finance.
c. Edge

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 [2024-06-26 17:36:40][DEBUG]: == [Content Writer] Task output: The Future of Artificial Intelligence: Trends, Key Players, and Noteworthy News

Introduction:
Artificial Intelligence (AI) is no longer a futuristic concept but a reality that is transforming various industries. From healthcare to finance, AI is making a significant impact on the way we live and work. In this article, we will explore the latest trends, key players, and noteworthy news in the world of AI.

Key Points:
1. Latest Trends:
a. Natural Language Processing (NLP): NLP is a subfield of AI that enables computers to understand and interpret human language. NLP is being used in various applications such as chatbots, virtual assistants, and customer service. For instance, Facebook's M chatbot can converse with humans on a wide range of topics, from weather forecasts to movie recommendations.
b. Reinforcement Learning: Reinforcement learning is a type of machine learning that allows computers to learn by interacting wit

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 [2024-06-26 17:43:00][DEBUG]: == [Editor] Task output: The Future of Artificial Intelligence: Trends, Key Players, and Noteworthy News

Introduction:
Artificial Intelligence (AI) is no longer a futuristic concept but a reality that is transforming various industries. From healthcare to finance, AI is making a significant impact on the way we live and work. In this article, we will explore the latest trends, key players, and noteworthy news in the world of AI.

Key Points:
1. Latest Trends:
a. Natural Language Processing (NLP): NLP is a subfield of AI that enables computers to understand and interpret human language. NLP is being used in various applications such as chatbots, virtual assistants, and customer service. For instance, Facebook's M chatbot can converse with humans on a wide range of topics, from weather forecasts to movie recommendations.
b. Reinforcement Learning: Reinforcement learning is a type of machine learning that allows computers to learn by interacting with their 

The Future of Artificial Intelligence: Trends, Key Players, and Noteworthy News

Introduction:
Artificial Intelligence (AI) is no longer a futuristic concept but a reality that is transforming various industries. From healthcare to finance, AI is making a significant impact on the way we live and work. In this article, we will explore the latest trends, key players, and noteworthy news in the world of AI.

Key Points:
1. Latest Trends:
a. Natural Language Processing (NLP): NLP is a subfield of AI that enables computers to understand and interpret human language. NLP is being used in various applications such as chatbots, virtual assistants, and customer service. For instance, Facebook's M chatbot can converse with humans on a wide range of topics, from weather forecasts to movie recommendations.
b. Reinforcement Learning: Reinforcement learning is a type of machine learning that allows computers to learn by interacting with their environment. Reinforcement learning is being used in various applications such as gaming, robotics, and finance. For example, Google's AlphaGo Zero, a computer program that can play the ancient Chinese board game Go at a superhuman level, has achieved a new milestone by teaching itself to play chess.
c. Edge Computing: Edge computing is a distributed computing paradigm that brings computation and data storage closer to the location where it is needed. Edge computing is being used in various applications such as autonomous vehicles, smart cities, and healthcare. For instance, BMW's iX3 electric SUV uses edge computing to process sensor data in real-time, enabling the vehicle to make quick decisions and respond to changing road conditions.

2. Key Players:
a. Google: Google is a leading player in the AI space with its AI subsidiary, DeepMind. DeepMind has made significant contributions to the field of AI, including developing AlphaGo, a computer program that can play the ancient Chinese board game Go at a superhuman level. Google's latest AI breakthrough is AlphaCode, a computer program that can solve complex mathematical problems in minutes, which would take a human mathematician several weeks to solve.
b. Amazon: Amazon is investing heavily in AI and